In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, AutoConfig
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import matplotlib.pyplot as plt

CHEMBERT_NAME = "seyonec/ChemBERTa-zinc-base-v1"
# Tokenizer

def tokenize(batch):
    return tokenizer(
        batch["smiles"],
        padding="max_length",
        truncation=True,
        max_length=160
    )

# or DeepChem/ChemBERTa-77M-MLM
tokenizer = AutoTokenizer.from_pretrained(CHEMBERT_NAME)
config = AutoConfig.from_pretrained(CHEMBERT_NAME,output_hidden_states=True)
encoder = AutoModel.from_pretrained(CHEMBERT_NAME,config=config)

df_train = pd.read_csv('./NIHDataset/train_df.csv')
df_val   = pd.read_csv('./NIHDataset/val_df.csv')
df_test  = pd.read_csv('./NIHDataset/test_df.csv')

df_train = df_train.rename(columns={"SMILES_CANON": "smiles", "logLD50": "LD50"})
df_val   = df_val.rename(columns={"SMILES_CANON": "smiles", "logLD50": "LD50"})
df_test  = df_test.rename(columns={"SMILES_CANON": "smiles", "logLD50": "LD50"})

df_train = df_train[['smiles', 'LD50']].dropna()
df_val   = df_val[['smiles', 'LD50']].dropna()
df_test  = df_test[['smiles', 'LD50']].dropna()
print(len(df_train), len(df_val), len(df_test))
train_ds = Dataset.from_pandas(df_train.reset_index(drop=True))
val_ds   = Dataset.from_pandas(df_val.reset_index(drop=True))
test_ds  = Dataset.from_pandas(df_test.reset_index(drop=True))

train_ds = train_ds.map(tokenize, batched=True)
val_ds   = val_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

train_ds = train_ds.rename_column("LD50", "labels")
val_ds   = val_ds.rename_column("LD50", "labels")
test_ds  = test_ds.rename_column("LD50", "labels")

cols = ["input_ids", "attention_mask", "labels"]
train_ds.set_format("torch", columns=cols)
val_ds.set_format("torch", columns=cols)
test_ds.set_format("torch", columns=cols)


def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.squeeze()

    mse = mean_squared_error(labels, preds)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(labels, preds)
    r2 = r2_score(labels, preds)

    return {
        "MSE": mse,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2
    }
def pool_hidden_states(outputs, attention_mask, mode="cls"):
    hidden_states = outputs.hidden_states

    if mode == "cls":
        return hidden_states[-1][:, 0]

    elif mode == "mean":
        last = hidden_states[-1]
        mask = attention_mask.unsqueeze(-1)
        return (last * mask).sum(1) / mask.sum(1)

    else:
        raise ValueError
class RegressionHead(nn.Module):
    def __init__(self, hidden_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.net(x)
class ChemBERTRegressor(nn.Module):
    def __init__(self, encoder, pooling="cls"):
        super().__init__()
        self.encoder = encoder
        self.pooling = pooling
        self.regressor = RegressionHead(
            hidden_dim=encoder.config.hidden_size
        )

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        pooled = pool_hidden_states(
            outputs, attention_mask, self.pooling
        )

        preds = self.regressor(pooled).squeeze(-1)

        loss = None
        if labels is not None:
            loss = nn.MSELoss()(preds, labels)

        return {"loss": loss, "logits": preds}

training_args = TrainingArguments(
    output_dir="./chembert_ld50",
    eval_strategy="epoch",   # FIXED
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="RMSE",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),
    report_to="none"
)
model = ChemBERTRegressor(
    encoder=encoder,
    pooling="cls"   # SAME as ModernBERT
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()
# ---- External test evaluation ----
test_outputs = trainer.predict(test_ds)

# Raw predictions & labels
test_preds  = test_outputs.predictions.squeeze()
test_labels = test_outputs.label_ids

# ---- Metrics ----
test_mse  = mean_squared_error(test_labels, test_preds)
test_rmse = np.sqrt(test_mse)
test_mae  = mean_absolute_error(test_labels, test_preds)
test_r2   = r2_score(test_labels, test_preds)

print("\n===== External Test Performance (ChemBERT) =====")
print(f"MAE  : {test_mae:.4f}")
print(f"RMSE : {test_rmse:.4f}")
print(f"R2   : {test_r2:.4f}")
print(f"MSE  : {test_mse:.4f}")
# Save predictions
test_results = pd.DataFrame({
    "SMILES": df_test["smiles"].values,
    "True_LD50": test_labels,
    "Pred_LD50": test_preds
})


def plot_with_metrics(df, true_col="True_LD50", pred_col="Pred_LD50"):
    
    y_true = df[true_col].values
    y_pred = df[pred_col].values

    # ---- Metrics ----
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))

    # ---- Fold errors ----
    fold_error = np.maximum(y_pred/y_true, y_true/y_pred)

    within_2fold = np.mean(fold_error <= 2) * 100
    within_3fold = np.mean(fold_error <= 3) * 100

    # ---- Plot ----
    min_val = min(y_true.min(), y_pred.min())
    max_val = max(y_true.max(), y_pred.max())
    x = np.linspace(min_val, max_val, 100)

    plt.figure(figsize=(7,7))

    plt.scatter(y_true, y_pred, alpha=0.7)

    # Ideal
    plt.plot(x, x, linestyle='--', linewidth=2, label="Ideal (y=x)")

    # 2-fold
    plt.plot(x, 2*x, linestyle=':', linewidth=2, label="±2-fold")
    plt.plot(x, x/2, linestyle=':', linewidth=2)

    # 3-fold
    plt.plot(x, 3*x, linestyle='-.', linewidth=2, label="±3-fold")
    plt.plot(x, x/3, linestyle='-.', linewidth=2)

    # ---- Text box ----
    textstr = (
        f"R² = {r2:.3f}\n"
        f"MAE = {mae:.3f}\n"
        f"RMSE = {rmse:.3f}\n\n"
        f"Within 2-fold = {within_2fold:.1f}%\n"
        f"Within 3-fold = {within_3fold:.1f}%"
    )

    plt.text(0.05, 0.95, textstr,
             transform=plt.gca().transAxes,
             fontsize=11,
             verticalalignment='top',
             bbox=dict(boxstyle="round", alpha=0.2))

    plt.xlabel("Actual LD50", fontsize=14)
    plt.ylabel("Predicted LD50", fontsize=14)
    plt.title("Prediction vs Actual (Test Set)", fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

    # Print metrics also
    print("R²:", r2)
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("Within 2-fold (%):", within_2fold)
    print("Within 3-fold (%):", within_3fold)
    
plot_with_metrics(test_results,true_col="True_LD50",pred_col="Pred_LD50")